In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/brettnaul/miniconda3/envs/deep/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [8]:
from keras import backend as K
#K.set_learning_phase(0)
%run asas_full.py 64 2 0.25 --model_type gru --lr 1e-3 --sim_type asas_full/n200 --nb_epoch 50 --n_min 200 --n_max 200 --m_max 32
# --filter_length 7 --batch_norm
# --embedding 32

Skipping PhasedLSTM...
['accuracy']
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/asas_full/n200/gru_064_x2_1m03_drop25
Train on 1828 samples, validate on 458 samples
Instructions for updating:
Please switch to tf.summary.merge_all.
Epoch 1/25
Epoch 1/25
1828/1828 [==============================] - 12s - loss: 1.0788 - acc: 0.5542 - val_loss: 0.9839 - val_acc: 0.5830
Epoch 2/25
Epoch 2/25
1828/1828 [==============================] - 12s - loss: 0.9888 - acc: 0.5979 - val_loss: 0.9428 - val_acc: 0.5852
Epoch 3/25
Epoch 3/25
1828/1828 [==============================] - 11s - loss: 0.9434 - acc: 0.6050 - val_loss: 0.9273 - val_acc: 0.5852
Epoch 4/25
Epoch 4/25
1828/1828 [==============================] - 11s - loss: 0.9159 - acc: 0.6050 - val_loss: 0.8876 - val_acc: 0.5852
Epoch 5/25
Epoch 5/25
1828/1828 [==============================] - 11s - loss: 0.8826 - acc: 0.6050 - val_loss: 0.8555 - val_acc: 0.5852
Epoch 6/25
Epoch 6/25
1828/1828 [==============================] - 11s - 

# Visualizing predictions

In [ ]:
%%time
pred = model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)}, batch_size=32)

In [ ]:
i = -1

In [ ]:
#inds = ~np.isnan(X_raw[i, :, 0])
inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = X_raw[i, inds, 0]
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
#plt.plot(t, pred[i], 'o');
plt.errorbar(t, pred[i], None, None, 'o');

#from gatspy.periodic import LombScargleFast
#gat_resids = np.zeros(len(X))
#model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.05 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
#model_gat.fit(t, m)
#print(model_gat.score(model_gat.best_period).item())

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original', 'Reconstructed'])
plt.title("MSE: {}, Var: {}".format((np.mean((m - np.squeeze(pred[i])) ** 2)), np.var(m)));

# Visualizing training progress

# New data

In [ ]:
np.random.seed(0)
train = np.sort(np.random.choice(np.arange(len(X)), int(len(X) * 0.8), replace=False))
valid = np.arange(len(X))[~np.in1d(np.arange(len(X)), train)]
y = np.argmax(Y, axis=1)
_, scale_params = preprocess(X_raw, args.m_max, True, True, True)

In [ ]:
encode_layer = [l for l in model.layers if l.name == 'encoding'][0]
inputs = [l for l in model.layers if 'Input' in str(l)]
encode = K.function([inputs[0].input, inputs[1].input], [encode_layer.output])
encoding, = encode([X, X[:, :, 0:1]])
encoding.shape

In [ ]:
encoding = np.c_[encoding, scale_params['means'], scale_params['scales']]

In [ ]:
i = -1

In [ ]:
inds = ~np.isnan(X_raw[i, :, 0])
#inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = X_raw[i, inds, 0]
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
#plt.plot(t, pred[i], 'o');
#plt.errorbar(t, pred[i], None, None, 'o');

from gatspy.periodic import LombScargleFast
gat_resids = np.zeros(len(X))
model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.001 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
model_gat.fit(t, m)
print("Best period: {} (Score: {})".format(model_gat.best_period, model_gat.score(model_gat.best_period).item()))

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original'])
plt.title("i={} ({})".format(i, classnames[y[i]]))

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

enc_train, enc_test, y_train, y_test = train_test_split(encoding, y, test_size=0.25, random_state=42)

#asas_model = RandomForestClassifier(n_estimators=128)
asas_model = GridSearchCV(RandomForestClassifier(), {'n_estimators': [100, 250], 'criterion': ['gini', 'entropy'], 'max_features': [0.1, 0.2, 0.3]})
#asas_model = SVC()
asas_model.fit(enc_train, y_train)

In [ ]:
print("Training score: {}%".format(100 * asas_model.score(enc_train, y_train)))
print("Test score: {}%".format(100 * asas_model.score(enc_test, y_test)))

# Direct classifier

In [11]:
%run asas 64 2 0.25 --model_type gru --lr 1e-3 --sim_type asas/n200 --nb_epoch 25 --n_min 200 --n_max 200 --m_max 32

Skipping PhasedLSTM...
['accuracy']
/Users/brettnaul/Dropbox/Documents/timeflow/keras_logs/asas/n200/gru_064_x2_1m03_drop25
Train on 1828 samples, validate on 458 samples
Instructions for updating:
Please switch to tf.summary.merge_all.
Epoch 1/25
Epoch 1/25
1828/1828 [==============================] - 12s - loss: 1.0936 - acc: 0.5542 - val_loss: 0.9296 - val_acc: 0.5895
Epoch 2/25
Epoch 2/25
1828/1828 [==============================] - 12s - loss: 0.9866 - acc: 0.5777 - val_loss: 0.9030 - val_acc: 0.5852
Epoch 3/25
Epoch 3/25
1828/1828 [==============================] - 12s - loss: 0.9243 - acc: 0.5990 - val_loss: 0.8293 - val_acc: 0.6026
Epoch 4/25
Epoch 4/25
1828/1828 [==============================] - 11s - loss: 0.8633 - acc: 0.6242 - val_loss: 0.7761 - val_acc: 0.6223
Epoch 5/25
Epoch 5/25
1828/1828 [==============================] - 11s - loss: 0.8192 - acc: 0.6504 - val_loss: 0.6929 - val_acc: 0.7183
Epoch 6/25
Epoch 6/25
1828/1828 [==============================] - 11s - loss:

# Light curve database